In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
import copy
import requests
import pickle
from io import BytesIO

In [21]:
def get_clf_eval(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred, average='macro')
    precision = precision_score(y_test, pred, average='macro')
    f1 = f1_score(y_test, pred, average='macro')
    print('Accuracy:{0:.4f}, Recall:{1:.4f}, Precision:{2:.4f}, F1-Score:{3:.4f}'.format(accuracy, recall, precision, f1))

In [ ]:
def make_confusion(y_test, y_pred):
    confusion = confusion_matrix(y_test,y_pred)
    confusion_df = pd.DataFrame(confusion,columns=['Predicted_1','Predicted_2', 'Predicted_3'],index=['Predicted_1','Predicted_2', 'Predicted_3'])
    
    return confusion_df

In [28]:
def load_dataset(directory):
    
    global input_cnt, output_cnt, data, x, y

    data = pd.read_csv(directory)
    input_cnt = data.loc[:, data.columns != 'Class Label'].shape[1]
    # output_cnt = data['Class Label'].shape[1]
    output_cnt = 1
    y = data['Class Label'].to_numpy()
    x = data.loc[:, data.columns != 'Class Label'].to_numpy()

In [ ]:
class DiscriminentAnalysis():
    def __init__(self, alpha=0.0, beta=0.0, eval_mode = False):
        if not eval_mode:
            self.learned = False
            self.alpha = alpha
            self.beta = beta
            self.class_names = []
            self.class_priors = {}
            self.class_means = {}
            self.regularized_covariances = {}
            self.rda_covariances = {}
            self.reset()

        else:
            self.load_parameter()


    def load_parameter(self):
        mLink = 'https://github.com/2U1/ML_RDACOV/blob/master/parameter.pkl?raw=true'
        mfile = BytesIO(requests.get(mLink).content)
        parameter = pickle.load(mfile)
        

        self.learned = True
        self.alpha = parameter['alpha']
        self.beta = parameter['beta']
        self.class_names = parameter['class_name']
        self.class_priors = parameter['calss_priors']
        self.class_means = parameter['class_means']
        self.regularized_covariances = parameter['reg_cov']
        self.rda_covariances = parameter['rda_cov']


    def reset(self):
        self.learned = False
        self.class_names = []
        self.class_priors = {}
        self.class_means = {}
        self.regularized_covariances = {}
        self.rda_covariances = {}


    def return_parameters(self):
        parameters = {
            'alpha': self.alpha,
            'beta': self.beta,
            'class_name': self.class_names,
            'calss_priors': self.class_priors,
            'class_means': self.class_means,
            'reg_cov': self.regularized_covariances,
            'rda_cov': self.rda_covariances
        }

        return parameters

    def fit(self, X, y):
        self.class_names = np.unique(y)
        class_covariances = {}
        pooled_covariances = 0
        for i in self.class_names:
            class_indices = np.where(y == i)[0]
            class_samples = X[class_indices, :]
            self.class_priors[i] = float(len(class_indices)) / len(y)
            self.class_means[i] = np.mean(class_samples, axis=0)
            class_covariances[i] = np.cov(class_samples, rowvar=0)
            pooled_covariances += class_covariances[i] * self.class_priors[i]
        # Calculate RDA regularized covariance matricies for each class
        for i in self.class_names:
            self.regularized_covariances[i] = (self.beta * pooled_covariances) + ((1 - self.beta) *class_covariances[i])
            # self.regularized_covariances[i] = (self.beta * class_covariances[i]) + ((1 - self.beta) * pooled_covariances)

        for i in self.class_names:
            self.rda_covariances[i] = ((1-self.alpha) * self.regularized_covariances[i]) + (self.alpha * (1/self.class_priors[i]) * np.trace(self.regularized_covariances[i]) * np.eye(self.regularized_covariances[i].shape[0]))
        
        self.learned = True
        return self

    def predict(self, x):
        if not self.learned:
            raise NameError('Fit model first')
        # Determine probability of each class given input vector
        
        class_prob = {}
        for i in self.class_names:
            # Divid the class delta calculation into 3 parts
            part1 = -0.5 * np.log1p(np.linalg.det(self.rda_covariances[i]))
            # part2 = -0.5 * np.dot(np.dot((x - self.class_means[i]).T, np.linalg.pinv(self.rda_covariances[i])), (x - self.class_means[i]))
            part2 = -0.5 * np.matmul(np.matmul((x - self.class_means[i]).T, np.linalg.pinv(self.rda_covariances[i])), (x - self.class_means[i]))
            part3 = np.log(self.class_priors[i])
            class_prob[i] = part1 + part2 + part3
        return max(class_prob, key=class_prob.get)

In [ ]:
class GridSearchRDA():
    def __init__(self, model, param_grid):
        self.model = model
        self.param_grid = param_grid
        self.alpha = 0
        self.beta = 0
        self.best_covariance = {}
        self.best_score = 0


    def fit(self, X, y, cv=2):
        
        # metric_score = []

        data_length = len(X)
        
        alpha_list = self.param_grid['alpha']
        beta_list = self.param_grid['beta']
        
        if data_length % cv == 0:
            cv_x = np.split(X, cv)
            cv_y = np.split(y, cv)
            
        else:
            remain = data_length % cv
            cv_x = np.split(X[:-remain], cv)
            cv_y = np.split(y[:-remain], cv)

        for alpha in alpha_list:
            for beta in beta_list:
                accuracy_score_list = []
                recall_score_list = []
                precision_score_list = []
                f1_score_list = []
                for i in range(cv):
                    self.model.reset()
                    self.model.alpha = alpha
                    self.model.beta = beta

                    test_x_cv = cv_x[i]
                    train_x_cv = np.vstack(cv_x[:i] + cv_x[i + 1:])

                    test_y_cv = cv_y[i]
                    train_y_cv = np.vstack(cv_y[:i] + cv_y[i + 1:]).flatten()

                    
                    self.model.fit(train_x_cv, train_y_cv)

                    pred = []

                    for data in test_x_cv:
                        pred.append(self.model.predict(data))
                    
                    accuracy_score_list.append(accuracy_score(test_y_cv, pred))
                    recall_score_list.append(recall_score(test_y_cv, pred, average='macro'))
                    precision_score_list.append(precision_score(test_y_cv, pred, average='macro'))
                    f1_score_list.append(f1_score(test_y_cv, pred, average='macro'))
                
                accuracy_mean_score = np.mean(np.array(accuracy_score_list))
                recall_mean_score = np.mean(np.array(recall_score_list))
                precision_mean_score = np.mean(np.array(precision_score_list))
                f1_mean_score = np.mean(np.array(f1_score_list))

                print("alpha:{0:.1f}, beta:{1:.1f}, accuracy:{2:.4f}, recall:{3:.4f}, precision:{4:.4f} ,f1-score:{5:.4f}"\
                    .format(alpha, beta, accuracy_mean_score ,recall_mean_score ,precision_mean_score ,f1_mean_score))

                if f1_mean_score > self.best_score:
                    self.best_score = f1_mean_score
                    self.alpha = alpha
                    self.beta = beta
                    self.best_covariance = copy.deepcopy(self.model.rda_covariances)
                    self.best_estimator = copy.deepcopy(self.model)

In [ ]:
def test_evaluation(test_x, test_y):

    model = DiscriminentAnalysis(eval_mode=True)


    prediction = []
    
    for testing in test_x:
        prediction.append(model.predict(testing))


    confusion = make_confusion(test_y, prediction)
    
    get_clf_eval(test_y, prediction)
    print('\n\n')
    print(confusion)